In [2]:
#data loader
#this will prepare training data for model

import tensorflow as tf
import os

In [3]:
from tensorflow.python.ops import array_ops, math_ops

In [4]:
#training the data
class dataloader(object):
    def _init__(self,image_dir,hr_image_size):
        self.image_paths = [os.path.jain(image_dir,x)for x in os.listdir(image_dir)]
        self.image_size=hr_image_size
#parse helps converting image to the machine understanding form
    def _parse_image(self,image_path):
        image = tf.io.read_file(image_path)
        image = tf.io.decode_jepg(image,channels=3)
        image = tf.image.conver_image_dtype(image,tf.float32)
        
        if tf.kerasbackend.image_data_format() == 'channels_last':
            shape= array_ops.shape(image)[:2]
        else:
            shape = array_ops_shape(image)[1:]
        cond = math_ops.reduce_all(shape >= tf.constant(self,image_size))
        
        image = tf.cond(cond,lambda: tf.identity(image)),lambda: tf.image.resize(image,[self.image_size,self.image_size])
        
        return image
    #random crop helps to randomly crop the image and helps in increase in accuracy in conversion
    def _random_crop(self,image):
        
        low_res =tf.image.randon_crop(image,[self.image_size,self.image_size,3])
        return image
    def _high_low_res_pairs(self , high_res):
        
        low_res=tf.image.resize(high_res, [self.image_size//4,self.image_size//4],method='bicubic')
        return low_res,high_res
    #rescale helps to zoom and unzoom the image for finding accuracy
    def _rescale(self , low_res,high_res):
        high_res = high_res * 2.0-1.0
        return low_res , high_res

In [4]:
def dataset(self, batch_size, threads=4):
    dataset = tf.data.Dataset.from_tensor_slices(self.image_paths)
    dataset = dataset.map(self._parse_image,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(self._random_crop,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(self._high_low_res_pairs,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.map(self._rescale,num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(30).batch(batch_size,drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
#creating the model architechure

In [2]:
from tensorflow import keras

In [3]:
class FastSRGAN(object):
    def _init_(self, args):
        self.hr_height=args.hr_size
        self.hr_width=args.hr_size
        self.lr_height=self.hr_height//4
        self.lr_width=self.hr_width//4
        self.lr_shape = (self.lr_height,self.lr_width, 3 )
        self.hr_shape = (self.hr_height,self.hr_width, 3 )
        self.iterations = 0
        
        self.n_residual_blocks=6
        
        self.gen_schedule = keras.optimizers.schedules.ExponentialDecay(args.lr, decay_steps=100000, decay_rate=0.1, staircase= True)
        
        self.disc_schedule = keras.optimizers.schedules.ExponentialDecay(args.lr*s, decay_steps=100000, decay_rate=0.1, staircase= True)
        
        self.gen_optimizer = keras.optimizers.Adam(learning_rate=self.gen_schedule)
        self.disc_optimizer = keras.optimizers.Adam(learning_rate=self.disc_schedule)
        
        self.vgg = self.build_vgg()
        self.vgg.trainable = False
        
        patch = int(self.hr_height/2**4)
        self.disc_patch = (patch,patch,1)
        
        self.gf = 32
        self.df = 32
        
        self.discriminator = self.build_discriminator()
        self.generator = self.build_generator()
        
    @tf.function
    def content_loss(self, hr , sr):
        sr = keras.application.vgg19.preprocess_input(((sr+1.0)*255)/2.0)
        hr = keras.application.vgg19.preprocess_input(((hr+1.0)*255)/2.0)
        sr_features = self.vgg(sr)/12.75
        hr_features = self.vgg(hr)/12.75
        return tf.keras.losses.MeanSquaredError()(hr_features, sr_features)
#using a pretrained model vgg19 which is trained for many images

    def build_vgg(self):
        vgg=keras.application.VGG19(weights='imagenet',input_shape.hr_shape,include_top=False)
        vgg.trainable= False
        
        for layer in vegg.layers:
            layer.trainable = False
            
        mode= keras.models.Model(input=vgg.input, outputs=vgg.get_layer("block5_conv4").output)
        
        return model
    def build_generator(self):
        def _make_divisible(v, divisor, min_value=None):
            if min_value is None:
                min_value = divisor
                
            new_v = max(min_value, int(v+divisor/2) // divisor * divisor)
            
            if new_v <0.9*v:
                new_v +=divisor
            return new_v
        def residual_block(inputs, filters, block_id, expansion=6, stride=1, alpha=1.0):
            channel_axis = 1 if keras.backed.image_data_format()=='channel_frist' else -1
            in_channels = keras.backend.int_shape(inputs)[channel_axis]
            
            pointwise_conv_filters = int(filters * alpha)
            pointwise_filters= _make_divisible(pointwise_conv_filters, 8)\
            x = inputs
            prefix = 'block_{}_' .format(block_id)
            
            if block_id:
                x = keras.layers.conv2D(expansion * in_channels, kernel_size = 1, padding ' same ',use_bias= True,activation=None,name=prefix +'expand')(x)
                x = keras.layers.BatchNormalization(axis=channel_axis, epsilon=1e-3,padding='same',use_bias=True , activation+'expand')(x)
                x = keras.layers.Activation('relu', name=prefix+'expand_relu')(x)
            else:
                prefix = 'expanded_conv_'
            x=keras.layers.DepthwiseConv2D(kernel_size=3,
                                          strides=stride,
                                          activation=None,
                                          use_bias=True,
                                          padding='same' if stride == 1 else 'valid',
                                          name= prefix+'depthwise')(x)
            
            x= keras.layers.BatchNormalization(axis=channel_axis,
                                              epsilon=1e-3,
                                              momentum=0.999,
                                              name=prefix+'depthwise_BN')(x)
            
            x = keras.layers.Activation('relu', name=prefix+'depthwise_relu')(x)
            
            x=keras.layers.conv2D(pointwise_filters,
                                 kernel_size=1,
                                 padding='same',
                                 activation=None,
                                 use_bias=True,
                                 name=prefix+'project')(x)
            
            x= keras.layers.BatchNormalization(axis=channel_axis,
                                              epsilon=1e-3,
                                              momentum=0.999,
                                              name=prefix+'project_BN')(x)
            if in_channel == poinwise_filters and stride == 1:
                return keras.layers.Add(name=prefix + 'add')([inputs,x])
            return x
        def deconv2D(layer_input, filters):
            
            u = keras.layers.conv2D(filters, kernel_size=3,strides=1, padding='same')(layer_input)
            u = tf.nn.depth_to_space(u,2)
            u = keras.layers.PReLU(shared_axes=[1,2])(u)
            return u
        img_lr = keras.Input(shape=self.lr_shape)
        
        c1=keras,layers.conv2D(self.gf,kernel_size=3,strides=1,padding='same')(img_lr)
        
        c1=keras,layers.BatchNormalization()(c1)
        
        c1=keras,layers.PReLU(shared_axes=[1,2])(c1)
        
        r=residual_block(c1,self.gf,0)
        for idx in range(1,self.n_residual_blocks):
            r=residual_block(r,self.gf,idx)
            
        c2=keras,layers.conv2D(self.gf,kernel_size=3,strides=1,padding='same')(r)
        c2=keras,layers.BatchNormalization()(c2)
        c2=keras,layers.Add([c1,c2])
        
        
        
        
        
        
        

SyntaxError: invalid syntax (1718486351.py, line 67)